In [395]:
import pandas as pd
import numpy as np
import re
from multiprocessing import Pool
import functools

data_path = 'Data/'

### Loading in test data
load data
organise info
clean up columns

In [396]:
choices = pd.read_csv(data_path + 'responses_so_far.csv')

In [397]:
agenda = [re.findall(r'\[([^\]]+)', col) for col in choices.columns[2:]]

In [398]:
regex = re.compile(".*?\((.*?)\)")
re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip())

'o'

In [399]:
new_cols = ['timestamp','name'] + [re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_') for co in agenda]

In [400]:
choices.columns = new_cols

In [401]:
schedule = [(co[0].replace(' ', '_'), re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_')) for co in agenda]

In [402]:
schedule_dict = {}
for pair in schedule:
    if pair[0] not in schedule_dict.keys():
        schedule_dict[pair[0]] = []
    schedule_dict[pair[0]].append(pair[1])

In [448]:
company_avai_dict_og = {re.sub("[\(\[].*?[\)\]]", "", co[1].split('[')[0].strip()).strip().replace(' ', '_'):int(co[1].split('[')[1]) for co in agenda}
company_avai_dict = company_avai_dict_og.copy()

### Random allocation

Create dictionary that matches time slots to choices

In [404]:
choices = choices.drop(['timestamp'], axis = 1)

Trying out allocation

In [405]:
# dropping NA
choices = choices[~choices.name.isnull()].reset_index(drop = True)

In [425]:
def prize_draw(co, level, pool):
    limit = company_avai_dict[co]
    new_pool = list(set(choices.name[choices[co] == level])&set(pool))
    if len(new_pool) > limit:
        return(list(np.random.choice(new_pool, limit, replace=False)))
    else:
        return(new_pool)

In [412]:
company_avai_dict_og

{'AQR': 15,
 'BNP_Paribas_Data_Analytics': 4,
 'Blockchain': 3,
 'CapGemini': 25,
 'Circle': 16,
 'Founders_Factory': 30,
 'Google_Deepmind': 8,
 'King.com': 4,
 'McLauren_Applied_Technologies': 20,
 'Neuroprofiler': 15,
 'Pierre_Richemond': 4,
 'PwC': 9,
 'Regnosys': 15,
 'Stéphane_Leroy': 4,
 'Vodafone_Lab': 25,
 'YOOBIC': 12,
 'Zopa': 15}

In [426]:
company_avai_dict = company_avai_dict_og
prize_draw_dict = {}
for slot, companies in schedule_dict.items():
    print('------------new time slot-------------')
    print(companies)
    pool = list(choices.name)
    all_levels = range(1,len(companies)+1)
    print(all_levels)
    for level in all_levels:
        for co in companies:
            print(co, level)
            if co not in prize_draw_dict.keys():
                prize_draw_dict[co] = []
            winners = prize_draw(co, level, pool)
            company_avai_dict[co] = company_avai_dict[co] - len(winners)
            prize_draw_dict[co].extend(winners)
            pool = [person for person in pool if person not in winners]
            print('there remains', len(pool), ' to be distributed')
            if company_avai_dict[co] == 0:
                print('company exhausted')
            if len(pool) == 0:
                print('pool exhausted')

------------new time slot-------------
['Neuroprofiler', 'Regnosys', 'AQR']
range(1, 4)
Neuroprofiler 1
there remains 26  to be distributed
company exhausted
Regnosys 1
there remains 21  to be distributed
AQR 1
there remains 7  to be distributed
Neuroprofiler 2
there remains 7  to be distributed
company exhausted
Regnosys 2
there remains 5  to be distributed
AQR 2
there remains 4  to be distributed
company exhausted
Neuroprofiler 3
there remains 4  to be distributed
company exhausted
Regnosys 3
there remains 0  to be distributed
pool exhausted
AQR 3
there remains 0  to be distributed
company exhausted
pool exhausted
------------new time slot-------------
['Google_Deepmind', 'Blockchain', 'Pierre_Richemond', 'Stéphane_Leroy', 'King.com', 'BNP_Paribas_Data_Analytics', 'Circle']
range(1, 8)
Google_Deepmind 1
there remains 33  to be distributed
company exhausted
Blockchain 1
there remains 33  to be distributed
Pierre_Richemond 1
there remains 31  to be distributed
Stéphane_Leroy 1
there re

In [420]:
for slot, companies in schedule_dict.items():
    total = sum([len(prize_draw_dict[company]) for company in companies])
    print('for', slot, total, ' have been allocated')

for TUESDAY_AFTERNOON 41  have been allocated
for WEDNESDAY_LUNCH 41  have been allocated
for WEDNESDAY_AFTERNOON 41  have been allocated
for THURSDAY_MORNING 41  have been allocated
for THURSDAY_AFTERNOON 41  have been allocated


In [421]:
len(choices)

41

Sanity Check

In [451]:
[len(winners) <= company_avai_dict[co] for co, winners in prize_draw_dict.items()]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

it seems as though we're good

### Save as xlsx

In [ ]:
pd.DataFrame.from_dict(prize_draw_dict, orient='index').transpose().to_excel(data_path+'prize_draw_res.xlsx')